In [1]:
import sys
if "convert" not in sys.path:
    #!!!path needs to change per user!!!
    sys.path.insert(0,"convert")
import pandas as pd

match = pd.read_csv("data/match.csv")
result = match["Score"].str.split("–",expand=True).rename({0:"Home Goal", 1:"Away Goal"},axis=1)

In [3]:
#twogk match
twogkmatch=[]
for idx in range(0,9027):
    homegk = pd.read_csv("data/stat/"+str(idx)+"_homegk.csv")
    awaygk = pd.read_csv("data/stat/"+str(idx)+"_awaygk.csv")
    if homegk.shape[0]+awaygk.shape[0]!=2:
        twogkmatch.append(idx)
#special player with gk values???
import pickle
from Classes import *

with open("data/namelist.pkl", "rb") as inp:
    nameli = pickle.load(inp)


In [3]:
nameli["Sevilla_Juan Sánchez"].stat.Pos="RW"
weird = []
for i, j in nameli.items():
    if j.stat["Pos"].value_counts().idxmax()!="GK" and j.stat.shape[1]>124:
        weird.append(i)


In [26]:
nameli["Barcelona_Lionel Messi"].stat

,Player,Nation,#,Pos,Age,Min,Gls,Ast,PK,PKatt,...,TklW.1,PKwon,PKcon,OG,Recov,Won,Lost.1,Won%,Date,Team
0,Lionel Messi,ar ARG,10.0,FW,30-057,90,0,0,0,0,...,0,0,0,0,1,0,0,NaN,2017-08-20,Barcelona
0,Lionel Messi,ar ARG,10.0,FW,30-063,90,2,0,0,1,...,0,0,0,0,2,0,0,NaN,2017-08-26,Barcelona
1,Lionel Messi,ar ARG,10.0,FW,30-077,90,3,0,0,0,...,1,0,0,0,2,0,0,NaN,2017-09-09,Barcelona
1,Lionel Messi,ar ARG,10.0,LW,30-084,90,0,1,0,0,...,0,0,0,0,0,0,0,NaN,2017-09-16,Barcelona
0,Lionel Messi,ar ARG,10.0,FW,30-087,90,4,0,1,1,...,0,0,0,0,1,0,0,NaN,2017-09-19,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,Lionel Messi,ar ARG,10.0,"FW,RW",33-309,90,1,0,0,0,...,0,0,0,0,3,0,0,NaN,2021-04-29,Barcelona
1,Lionel Messi,ar ARG,10.0,FW,33-312,90,2,0,0,1,...,0,0,0,0,1,0,0,NaN,2021-05-02,Barcelona
1,Lionel Messi,ar ARG,10.0,FW,33-318,90,0,0,0,0,...,1,0,0,0,2,0,0,NaN,2021-05-08,Barcelona
0,Lionel Messi,ar ARG,10.0,FW,33-321,90,1,0,0,0,...,0,0,0,0,0,0,1,0.0,2021-05-11,Barcelona


In [5]:
for i in weird:
    nameli[i].stat = nameli[i].stat[nameli[i].stat.Pos!="GK"].iloc[:,:124]

In [6]:
n = match.shape[0]
finalstatmatrix= pd.DataFrame()
start= 0
for idx in range(0,9027):
    if idx in twogkmatch:
        continue
    date = match.iloc[idx,:]["Date"]
    home = match.iloc[idx,:]["Home"]
    away = match.iloc[idx,:]["Away"]
    players = []
    #find names of top 11 players who played longest on field
    homestat = pd.read_csv("data/stat/"+str(idx)+"_homestat.csv")
    awaystat = pd.read_csv("data/stat/"+str(idx)+"_awaystat.csv")
    homegk = pd.read_csv("data/stat/"+str(idx)+"_homegk.csv")
    awaygk = pd.read_csv("data/stat/"+str(idx)+"_awaygk.csv")
    # Order in which Players added to the list MATTERS!!! (Each column corresponds to a unique weight position)
    players.extend(homestat.sort_values("Min",ascending=False).head(10).Player)
    players.extend(homegk.sort_values("Min",ascending=False).head(1).Player)
    players.extend(awaystat.sort_values("Min",ascending=False).head(10).Player)
    players.extend(awaygk.sort_values("Min",ascending=False).head(1).Player) #should get 22 player's name by now
    for i in range(len(players)):
        if i<=10:
            players[i] = home+"_"+players[i]
        else:
            players[i] = away+"_"+players[i]
        
    finalstatvector= pd.Series(dtype='float64')
    for name in players: # go get past 5 result
        try:
            temp = nameli[name].stat
            average_performance = temp[temp["Date"]<= date].sort_values("Date",ascending=False).iloc[:,5:].drop(columns=["Date","Team"]).head(5).mean()
            finalstatvector= pd.concat([finalstatvector,average_performance],axis=0)
        except Exception:
            raise Exception("Players not found in pkl")
    if idx== start:
        finalstatmatrix = finalstatvector
    else:
        print(finalstatmatrix.shape,idx,finalstatvector.shape)
        finalstatmatrix = pd.concat([finalstatmatrix,finalstatvector],axis=1)
    
finalstatmatrix.to_csv("data/modelmatrix.csv")

(2614,) 1 (2614,)
(2614, 2) 2 (2614,)
(2614, 3) 3 (2614,)
(2614, 4) 4 (2614,)
(2614, 5) 5 (2614,)
(2614, 6) 6 (2614,)
(2614, 7) 7 (2614,)
(2614, 8) 8 (2614,)
(2614, 9) 9 (2614,)
(2614, 10) 10 (2614,)
(2614, 11) 11 (2614,)
(2614, 12) 12 (2614,)
(2614, 13) 13 (2614,)
(2614, 14) 14 (2614,)
(2614, 15) 15 (2614,)
(2614, 16) 16 (2614,)
(2614, 17) 17 (2614,)
(2614, 18) 18 (2614,)
(2614, 19) 19 (2614,)
(2614, 20) 20 (2614,)
(2614, 21) 21 (2614,)
(2614, 22) 22 (2614,)
(2614, 23) 23 (2614,)
(2614, 24) 24 (2614,)
(2614, 25) 25 (2614,)
(2614, 26) 26 (2614,)
(2614, 27) 27 (2614,)
(2614, 28) 28 (2614,)
(2614, 29) 29 (2614,)
(2614, 30) 31 (2614,)
(2614, 31) 32 (2614,)
(2614, 32) 33 (2614,)
(2614, 33) 34 (2614,)
(2614, 34) 35 (2614,)
(2614, 35) 36 (2614,)
(2614, 36) 38 (2614,)
(2614, 37) 39 (2614,)
(2614, 38) 40 (2614,)
(2614, 39) 41 (2614,)
(2614, 40) 42 (2614,)
(2614, 41) 43 (2614,)
(2614, 42) 44 (2614,)
(2614, 43) 45 (2614,)
(2614, 44) 46 (2614,)
(2614, 45) 47 (2614,)
(2614, 46) 48 (2614,)
(2614, 4

In [8]:

finalstatmatrix = finalstatmatrix.transpose()
finalstatmatrix.reset_index(inplace==True)
finalstatmatrix

,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,...,Launch%,AvgLen,Att.2.1,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,AvgDist
0,90.0,1.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,34.0,...,100.00,56.80,11.0,100.00,67.80,13.0,0.0,0.000,2.0,44.500
1,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,...,31.40,31.20,7.0,57.10,42.40,13.0,3.0,23.100,0.0,12.200
0,90.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,28.0,...,66.70,52.30,10.0,100.00,72.20,23.0,2.0,8.700,1.0,15.600
0,90.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,30.0,...,19.00,30.00,10.0,70.00,51.20,12.0,0.0,0.000,0.0,17.000
0,90.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,72.0,...,72.40,52.70,10.0,90.00,65.10,19.0,0.0,0.000,2.0,14.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,85.6,0.0,0.2,0.0,0.0,2.2,0.8,0.4,0.0,53.8,...,22.76,27.60,5.6,27.82,29.44,12.8,2.0,13.575,1.4,13.700
0,47.4,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,11.0,...,34.06,33.44,11.8,42.96,35.34,15.8,1.8,17.200,1.8,11.440
0,89.6,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,52.6,...,23.36,27.82,5.8,30.08,30.38,9.0,0.6,10.660,0.8,18.025
0,75.8,0.8,0.0,0.0,0.0,2.2,1.2,0.0,0.0,45.2,...,34.44,33.22,7.4,54.66,39.48,12.2,0.2,1.820,1.0,14.700


In [23]:
finalstatmatrix = pd.read_csv("data/modelmatrix.csv")
finalstatmatrix.set_index('Unnamed: 0',inplace=True)
finalstatmatrix = finalstatmatrix.transpose()
finalstatmatrix

Unnamed: 0,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,...,Launch%,AvgLen,Att.2.1,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,AvgDist
0,90.0,1.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,34.0,...,100.00,56.80,11.0,100.00,67.80,13.0,0.0,0.000,2.0,44.500
1,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.0,...,31.40,31.20,7.0,57.10,42.40,13.0,3.0,23.100,0.0,12.200
0.1,90.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,28.0,...,66.70,52.30,10.0,100.00,72.20,23.0,2.0,8.700,1.0,15.600
0.2,90.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,30.0,...,19.00,30.00,10.0,70.00,51.20,12.0,0.0,0.000,0.0,17.000
0.3,90.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,72.0,...,72.40,52.70,10.0,90.00,65.10,19.0,0.0,0.000,2.0,14.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.8817,85.6,0.0,0.2,0.0,0.0,2.2,0.8,0.4,0.0,53.8,...,22.76,27.60,5.6,27.82,29.44,12.8,2.0,13.575,1.4,13.700
0.8818,47.4,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,11.0,...,34.06,33.44,11.8,42.96,35.34,15.8,1.8,17.200,1.8,11.440
0.8819,89.6,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,52.6,...,23.36,27.82,5.8,30.08,30.38,9.0,0.6,10.660,0.8,18.025
0.8820,75.8,0.8,0.0,0.0,0.0,2.2,1.2,0.0,0.0,45.2,...,34.44,33.22,7.4,54.66,39.48,12.2,0.2,1.820,1.0,14.700


Unnamed: 0,index,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,...,Launch%,AvgLen,Att.2.1,Launch%.1,AvgLen.1,Opp,Stp,Stp%,#OPA,AvgDist
0,0,90.0,1.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,...,100.00,56.80,11.0,100.00,67.80,13.0,0.0,0.000,2.0,44.500
1,1,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,31.40,31.20,7.0,57.10,42.40,13.0,3.0,23.100,0.0,12.200
2,0.1,90.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,...,66.70,52.30,10.0,100.00,72.20,23.0,2.0,8.700,1.0,15.600
3,0.2,90.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,19.00,30.00,10.0,70.00,51.20,12.0,0.0,0.000,0.0,17.000
4,0.3,90.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,72.40,52.70,10.0,90.00,65.10,19.0,0.0,0.000,2.0,14.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8818,0.8817,85.6,0.0,0.2,0.0,0.0,2.2,0.8,0.4,0.0,...,22.76,27.60,5.6,27.82,29.44,12.8,2.0,13.575,1.4,13.700
8819,0.8818,47.4,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,...,34.06,33.44,11.8,42.96,35.34,15.8,1.8,17.200,1.8,11.440
8820,0.8819,89.6,0.0,0.0,0.0,0.0,0.2,0.2,0.2,0.0,...,23.36,27.82,5.8,30.08,30.38,9.0,0.6,10.660,0.8,18.025
8821,0.8820,75.8,0.8,0.0,0.0,0.0,2.2,1.2,0.0,0.0,...,34.44,33.22,7.4,54.66,39.48,12.2,0.2,1.820,1.0,14.700
